In [7]:
main = putStr "Age? " >> getLine >>= \a -> if read a > 40 then putStrLn "too old" else putStrLn "nice!"

main2 = do
    putStr "Age? "
    a <- getLine
    if read a > 40 then 
        putStrLn "too old" 
    else 
        putStrLn "nice!"

`(IO (), >>, return ())` ist ein Monoid
aber `IO String` ist 
```haskell
return () >> act = act
act >> return () = act
```

`(IO, >>=, return())` ist eine Monade (da der bind operator noch einen wert zurück gibt, also eine funktion ist)
```haskell 
>>=     :: IO a -> (a -> IO b) -> IO b
return  :: a -> IO a
return x >>= f    = f x
act >>= return    = act
(act >>= f) >>= g = act >>= (\x -> fx) >>= g
```


Frage: ist (Maybe, +1, Nothing) eine Monade?
Nein, weil die funktion in der Mitte vom Typ IO a -> usw (siehe >>=) sein muss

## Zeigen, dass Monaden gebraucht werden

In [8]:
data Exp = Num Int
    | Exp :+: Exp
    | Exp :/: Exp
    deriving Show

e1 = Num 42 :+: Num 31
e2 = e1 :/: (e1 :+: Num (-73))

eval :: Exp -> Maybe Int
eval (Num n) = Just n
eval (e1 :+: e2) = 
    case eval e1 of
        Just v1 -> case eval e2 of 
            Just v2 -> Just (v1 + v2)
            Nothing -> Nothing
        Nothing -> Nothing
eval (e1 :/: e2) = 
    case eval e2 of
        Just 0 -> Nothing
        Just v2-> case eval e1 of 
            Just v1 -> Just (v1 `div` v2)
            Nothing -> Nothing
        Nothing -> Nothing

## nun gleicher code mit Monade

`Maybe` ist eine Fehlermodade, dh Nothing ist der Fehler

`Just x` ist der Erfolg mit dem Rückgabewert x

In [9]:
eval (Num n) = return n
eval (e1 :+: e2) = do
    v1 <- eval e1
    v2 <- eval e2
    return (v1 + v2)
eval (e1 :/: e2) = do
    v2 <- eval e2 
    if v2 == 0 then
        Nothing
    else do
        v1 <- eval e1
        return (v1 `div` v2)

In [10]:
class Monad m where
    return  :: a -> m a
    (>>)    :: m a -> m b -> m b
    (>>=)   :: m a -> (a -> m b) -> m b
    mx >> my = mx >>= \_ -> my

instance Monad Maybe where
    -- return :: a -> Maybe a
    return = Just 
    -- (>>=) Maybe a -> (a -> Maybe b) -> Maybe b
    Nothing >>= _ = Nothing 
    Just x >>= f  = f x


Line 5: Use >>
Found:
mx >>= \ _ -> my
Why not:
mx >> myLine 5: Use const
Found:
\ _ -> my
Why not:
const my

In [11]:
Just 41 >>= \x -> return (x + 1) -- Just 42
Nothing >>= \x -> return (x + 1) -- Nothing

Line 2: Evaluate
Found:
Nothing >>= \ x -> return (x + 1)
Why not:
Nothing

Just 42

Nothing

## Fmap für Monaden

In [12]:
genfmap :: Monad m => (a -> b) -> (m a -> m b)
genfmap  f mx = mx >>= \x -> return (f x)

```
 Functor f     => fmap  :: (a -> b) -> f a -> f b
     |
Applicative f  => (<*>) :: f (a -> b) -> f a -> f b
     |         => pure  :: a -> f a
     | 
  Monad m
```


In [13]:
class Functor f => Applicative f where
    pure :: a -> f a
    (<*>) :: f (a -> b) -> f a -> f b
    -- (*>) :: f a -> f b -> f b
    -- ax *> ay = pure (\_ x -> x) <*> ay <*> ax
    -- (<*) :: f a -> f b -> f a
    -- ax <* ay = pure (\_ x -> x) <*> ax <*> ay

instance Applicative Maybe where
    (<*>) :: Maybe (a -> b) -> Maybe a -> Maybe b
    (<*>) (Just f) (Just x) = Just (f x)
    (<*>) _ _ = Nothing
-- weitere gesetze die später dazu gekommen sind

In [14]:
eval (Num n) = return n
eval (e1 :+: e2) = do
    pure (+) <*> eval e1 <*> eval e2
eval (e1 :/: e2) = 
    case eval e2 of
        Just 0 -> Nothing 
        mv2    -> pure (flip div) <*> mv2 <*> eval e1

Line 3: Use <$>
Found:
pure (+) <*> eval e1
Why not:
((+) <$> eval e1)Line 7: Use <$>
Found:
pure (flip div) <*> mv2
Why not:
(flip div <$> mv2)

```haskell
(pure (+) <*> eval e1) <*> eval e2
      (+) :: Int -> Int -> Int
 pure (+) :: Maybe (Int -> Int -> Int)
          <*> :: Maybe (a -> b) -> Maybe a -> Maybe b
              eval e1  :: Maybe Int
(pure (+) <*> eval e1) :: Maybe (Int -> Int)
                           eval e2  :: Maybe Int
(pure (+) <*> eval e1) <*> eval e2  :: Maybe Int
```

In [15]:
genfmap :: Applicative f => (a -> b) -> f a -> f b
genfmap g fx = pure g <*> fx

Line 2: Use <$>
Found:
pure g <*> fx
Why not:
g <$> fx

In [16]:
data Tree a = Empty 
        | Branch (Tree a) a (Tree a)
        deriving Show

toList :: Tree a -> [a]
toList Empty = []
toList (Branch l v r) = toList l ++ [v] ++ toList r

In [17]:
largeTree :: Int -> Tree Int
largeTree 0 = Empty
largeTree n = Branch (largeTree (n - 1)) n (largeTree (n - 1))

largeUnbalancedTree :: Int -> Tree Int
largeUnbalancedTree 0 = Empty
largeUnbalancedTree n = Branch (largeUnbalancedTree (n - 1)) n Empty

t3 = largeTree 3
t20 = largeTree 20
t10k = largeUnbalancedTree 10000
t20k = largeUnbalancedTree 20000

In [26]:
length $ toList t10k
-- braucht 1 s

10000

In [35]:
length $ toList t20k
-- braucht 4s

Could not parse: (+s)

20000

In [38]:
naivReverse :: [a] -> [a]
naivReverse [] = []
naivReverse (x:xs) = naivReverse xs ++ [x]

In [39]:
rev :: [a] -> [a]
rev xs = rev' xs [] where
    rev' :: [a] -> [a] -> [a]
    rev' [] acc = acc
    rev' (x:xs) acc = rev' xs (x:acc)

**das jetzt schöner**

mithilfe von functional Lists: 
```haskell
[] = id
(++) = (.)

In [40]:
rev :: [a] -> [a]
rev xs = rev' xs [] where
    rev' :: [a] -> ([a] -> [a])
    rev' [] = id
    rev' (x:xs) = rev' xs . (x:)

**jetzt definieren wir datentypen die das alles verstecken**

In [41]:
newtype FList a  = FList ([a] -> [a])

toFList :: [a] -> FList a
toFList xs = FList (xs ++)

fromFList :: FList a -> [a]
fromFList (FList fs) = fs []

instance Semigroup (FList a) where
    -- <> :: FList a -> FList a -> FList a
    (<>) (FList xs) (FList ys) = FList (xs . ys)

instance Monoid (FList a) where
    mempty = FList id


In [48]:
frev :: [a] -> [a]
frev xs = fromFList (rev' xs) where
    rev' :: [a] -> FList a
    rev' [] = mempty
    rev' (x:xs) = rev' xs `mappend` toFList [x]